In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.arange(12).reshape(3, 4),
                  columns=['col_0', 'col_1', 'col_2', 'col_3'],
                  index=['row_0', 'row_1', 'row_2'])
print(df)

       col_0  col_1  col_2  col_3
row_0      0      1      2      3
row_1      4      5      6      7
row_2      8      9     10     11


In [3]:
a=[1, 1, 1]
for i in range(len(a)):
    print("w")

w
w
w


In [1]:
import glob
import os
import numpy as np
import torch
import torch.optim as optim
import torchvision.transforms.functional as TF
from torchvision.transforms import v2
import torchvision.models as models
import torch.nn as nn
import cv2
from torch.utils.data import Dataset
from pathlib import Path
from typing import List, Tuple
from torch.utils.data import DataLoader

date=20240513
time=1528
filedate=str(date)+"_"+str(time)
print(filedate)

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
cpu = torch.device('cpu')

Channels=3
IMG_SIZE=224
target_epoch=120

Classes = ["bread1", "bread2", "bread3"]
ClassNum = len(Classes)

testpath=r".\Forward"
savepath=r".\TrainedWeights"

#jpgしか読み込めません
data=list(Path(testpath).glob("*.jpg"))
if(len(data)<1):
    data=list(Path(testpath).glob("*.png"))
print(data)

'''
PytorchではDataloaderという,膨大なデータセットからでもメモリを圧迫せずに取り出せてforループにも対応するための枠組みがある
データセットをDataloaderが引っ張ってこれるような形式にするためにMyDataset(torch.utils.data.Dataset)というクラスを作れば，
あとはそのメソッドをtorch.utils.data.Datasetが勝手に使用してデータを加工してくれる
__init__, __getitem__, __len__をクラス内で必ず定義しなければならない
Dataloader内のデータはバッチごとにまとめられる
'''

class MyDataset(Dataset):
    def __init__(self, data, transforms, Classes) -> None:
        super().__init__()
        self.transforms = transforms
        self.Classes = Classes
        #globは複数のファイルのパスをまとめて取得する
        #訓練と訓練白黒の二個下のディレクトリから画像を取得
        self.data = data

    # ここで取り出すデータを指定している
    def __getitem__(
            self,
            index: int
    ) -> Tuple[torch.Tensor, torch.Tensor]:

        data = self.data[index]
        img1 = cv2.imread(str(data))
        img1 = cv2.resize(img1, (IMG_SIZE, IMG_SIZE))
        img1 = TF.to_tensor(img1)

        # データの変形 (transforms)
        transformed_img = self.transforms(img1)

        return transformed_img

    # この method がないと DataLoader を呼び出す際にエラーを吐かれる
    def __len__(self) -> int:
        return len(self.data)

#入力データに施す処理
transforms = v2.Compose([
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0,0,0], std=[0.2, 0.2, 0.2]),
        ])

testset= MyDataset(data=data, transforms=transforms, Classes=Classes)


testloader = DataLoader(dataset=testset,batch_size=len(testset),shuffle=False)
print("testloader length:", len(testloader))


resnet50 = models.resnet50()

#modify first layer so it expects 4 input channels; all other parameters unchanged
resnet50.conv1 = torch.nn.Conv2d(Channels,64,kernel_size = (7,7),stride = (2,2), padding = (3,3), bias = False)
#modifying final layer
resnet50.fc = nn.Linear(2048,ClassNum)

#lossfunction&optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.9)

def evaluate(testloader, model, loss_fn, optimizer):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    model.eval()
    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X in testloader:
            X=X.to(device)
            pred = model(X)
            
    pred=pred.to(cpu)

    #predは各クラスの確率になってる（onehotに近い）ので実際のクラス番号に戻す
    pred_class=pred.argmax(1)
    pred_class=np.array(pred_class)
    print("predicted class:", pred_class)

#モデル構築
modelpath = Path(savepath+"\\"+str(target_epoch)+"\model_weights"+filedate+".pth")
epochmodel = resnet50
epochmodel.load_state_dict(torch.load(modelpath))
#GPUにニューラルネットワークを渡す
epochmodel=epochmodel.to(device)

print("Model in Epoch", target_epoch)
#テストデータで評価
evaluate(testloader, epochmodel, loss_fn, optimizer)

print('Forwarding Complete!!!')

20240513_1528
[WindowsPath('Forward/bread20000.jpg')]
testloader length: 1
Model in Epoch 120
predicted class: [1]
Forwarding Complete!!!
